In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


     |██████████▋                     | 1.2 MB 23.4 MB/s eta 0:00:01

     |█████████████████████▌          | 2.4 MB 23.4 MB/s eta 0:00:01

     |████████████████████████████████| 3.5 MB 23.4 MB/s 


     |████████████████████████████████| 67 kB 32.2 MB/s 


...

     |████████████████████▏           | 153 kB 22.2 MB/s eta 0:00:01

     |████████████████████████████████| 243 kB 22.2 MB/s 


     |██████▉                         | 348 kB 29.8 MB/s eta 0:00:01

     |██████████████████████████████▎ | 1.5 MB 29.8 MB/s eta 0:00:01

     |████████████████████████████████| 1.6 MB 29.8 MB/s 


In [ ]:
import json
from datasets import load_dataset, load_metric, load_from_disk
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AlbertTokenizer, AlbertModel
from transformers import Adafactor
import torch
from torch import nn
import torch.nn.functional as F
import collections
from typing import List
from sklearn.metrics import f1_score

In [ ]:
class DataClass:

  def __init__(self, data_dir):
    self.data_dir = data_dir

  def data_csv(self, f, output):

    answers = []
    rewrites = []
    passages = []

    filepath = self.data_dir+f

    with open(filepath) as fl:
      data = json.load(fl)
      
      for d in data:
        answers.append(d['answer'])
        rewrites.append(d['rewrite'])
        passages.append(d['passage'])

      data = {'answer':answers, 'passage':passages, 'rewrite':rewrites}
      df = pd.DataFrame(data)
      df.to_csv(output, index=False)


data = DataClass('/storage/qrecc/')

data.data_csv('qrecc_train.json', 'train.csv')
data.data_csv('qrecc_test.json', 'test.csv')

qrecc = load_dataset('csv', data_files={'train': 'train.csv', 'test': 'test.csv'})

Using custom data configuration default-90c445d77f7c2491


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-90c445d77f7c2491/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#@title models and hyperparameters

max_length= 384
batch_size = 4
pretrained_model = 't5-base'

t5_tokenizer = T5Tokenizer.from_pretrained(pretrained_model)

t5_model = T5ForConditionalGeneration.from_pretrained(pretrained_model)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
def tokenize_dataset(batch):

  passages = t5_tokenizer(batch['rewrite'], batch['passage'], padding='max_length', truncation='only_second', max_length=max_length, add_special_tokens=True)
  answers = t5_tokenizer(batch['answer'], padding='max_length', truncation=True, max_length=max_length, add_special_tokens=True)

  batch['psg_input_ids'] = passages.input_ids
  batch['ans_input_ids']  = answers.input_ids
  batch['psg_attention_mask'] = passages.attention_mask

  return batch


# handle examples with no answers
def no_ans(x):
  if isinstance(x['answer'], str): return x
  x['answer'] = 'no_ans'
  return x


# removing examples with no context
qrecc = qrecc.filter(lambda x: isinstance(x['passage'], str))

# removing examples with passage length > 384
#qrecc = qrecc.filter(lambda x: len(e_tokenizer(x['passage']).input_ids) <= max_length)

# no answers
qrecc = qrecc.map(no_ans)

# tokenizing
dataset = qrecc.map(
    tokenize_dataset, 
    batch_size = batch_size,
    batched=True,
    remove_columns=['passage', 'answer', 'rewrite']
)


dataset.set_format(
    type='torch', columns=['psg_input_ids', 'ans_input_ids', 'psg_attention_mask'],)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

  0%|          | 0/6010 [00:00<?, ?ba/s]

  0%|          | 0/1620 [00:00<?, ?ba/s]

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset['train'], batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset['test'], batch_size=batch_size)

In [ ]:
def valid_loss():
  
  val_loss = 0
  idx = 0

  for batch in test_loader:
    # rewrite+passge inputs for encoder
    psg_input = batch['psg_input_ids'].to(device)
    psg_attention = batch['psg_attention_mask'].to(device)

    # answer inputs for rc
    ans_input = batch['ans_input_ids'] 
    ans_input[ans_input == t5_tokenizer.pad_token_id] = -100 # tokens with indices set to -100 are ignored (masked)
    ans_input = ans_input.to(device)

    # rc loss
    loss = t5_model(input_ids=psg_input, attention_mask=psg_attention, labels=ans_input).loss

    val_loss += loss.item()

    del psg_input, psg_attention, ans_input, loss

    idx += 1

  return val_loss/idx

In [ ]:
num_epochs = 3

device = torch.device('cuda')
t5_model.to(device)

t5_model.load_state_dict(torch.load('/storage/qrecc/models/rc/rc_gen4.pth'))

t5_model.train()


optim = optimizer = Adafactor(
    t5_model.parameters(),
    lr=1e-5,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False
)

for epoch in range(num_epochs):
  
  epoch_loss = 0

  for batch in train_loader:

    # rewrite+passge inputs for encoder
    psg_input = batch['psg_input_ids'].to(device)
    psg_attention = batch['psg_attention_mask'].to(device)

    # answer inputs for rc
    ans_input = batch['ans_input_ids'] 
    ans_input[ans_input == t5_tokenizer.pad_token_id] = -100
    ans_input = ans_input.to(device)

    # rc loss
    loss = t5_model(input_ids=psg_input, attention_mask=psg_attention, labels=ans_input).loss

    epoch_loss += loss.item()

    loss.backward()
    optim.step()
    optim.zero_grad()

    del psg_input, psg_attention, ans_input, loss

  print('Train loss after epoch {} : {}'.format(epoch+5, epoch_loss / len(train_loader)))
  t5_model.eval()
  print('Valid loss after epoch {} : {}'.format(epoch+5, valid_loss()))
  print('\n')
  t5_model.train()
  torch.save(t5_model.state_dict(), '/storage/qrecc/models/rc/rc_gen'+str(epoch+5)+'.pth')

Train loss after epoch 5 : 0.46682285451037364


Valid loss after epoch 5 : 0.5254644630971238




Train loss after epoch 6 : 0.445149001662934


Valid loss after epoch 6 : 0.5261607296914322




Train loss after epoch 4 : 0.49289952048291136


Valid loss after epoch 4 : 0.5289757966052419

In [ ]:
device = torch.device('cuda')
t5_model.to(device)

t5_model.load_state_dict(torch.load('/storage/qrecc/rc_inf3.pth'))
t5_model.eval()

<All keys matched successfully>

In [ ]:
device = torch.device('cuda')
t5_model.to(device)

t5_model.load_state_dict(torch.load('/storage/qrecc/models/e2e/rc3.pth'))
t5_model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [ ]:
def compute_f1_from_tokens(gold_toks: List[str], pred_toks: List[str]) -> float:
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())

  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)

  if num_same == 0:
    return 0

  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

idx = 0
f1 = 0

for batch in test_loader:

  # rewrite+passge inputs for encoder
  psg_input = batch['psg_input_ids'].to(device)
  psg_attention = batch['psg_attention_mask'].to(device)
  ans_input = batch['ans_input_ids'] 

  outputs = t5_model.generate(input_ids=psg_input, attention_mask=psg_attention)
  output_text = t5_tokenizer.batch_decode(outputs, skip_special_tokens=True)
  true_text = t5_tokenizer.batch_decode(ans_input, skip_special_tokens=True)

  for i in range(len(output_text)):
    score = compute_f1_from_tokens(true_text, output_text)
    #if score < 0: score = 0
    f1 += score
    idx += 1

print(f1 / idx)

0.3013275702377277


0.3173819079962952